In [1]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import math
import scipy.sparse as sp
from copy import deepcopy
import time
from sklearn.metrics import roc_auc_score
import sys
import multiprocessing as mp

import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torch_geometric
from torch_geometric.nn import GCNConv

from utils import *
from models import *

SEED = 42
np.random.seed(SEED)
random.seed(SEED)

import warnings
warnings.filterwarnings('ignore')
# %matplotlib inline

In [2]:
mixed_model = 'both' # GMF NMF GNN both
gnn = 'GCN' # GCN SAGE GAT GIN
mf = 'NMF' # NMF MLP NMF+MLP
drop_ratio = 0.5
num_negatives = 1
num_layers = 3
num_mlp = 4

In [3]:
data_name = 'usair'
num_sample = 332
ratio_sample_pos = 20
feature_pre = False
batch_size = 512

#################Parameters for model#################
loc_time = time.localtime()
model_config={'num_users': num_sample,
              'num_epoch': 1000,
              'batch_size': batch_size,
              'optimizer': 'adam', # adam sgd
              'adam_lr': 1e-4,
              'l2_regularization': 0, # 0.0000001,  # MLP model is sensitive to hyper params,
              'num_negative': 1,
              'drop_ratio': 0.5,
              'mlp_layers': [64, 128, 64, 32, 16],  # layers[0] is the concat of latent user vector & latent item vector
              'verbose': 100, 
              'feature_pre': False,
              'gnn': gnn,
              'mf': mf,
              'mixed_model': mixed_model,
              'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model',
              'alias':'test_model_{}_{}_{}_{}_{}'.format(loc_time[0], loc_time[1], loc_time[2], loc_time[3], loc_time[4])}

model_config['gnn_layers'] = [256, 128, 64, 32]

model_config['use_cuda'] = torch.cuda.is_available()

print('Parameters of Model are:')
for _ in model_config:
    print(_, model_config[_])
    
# set up device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Parameters of Model are:
num_users 332
num_epoch 1000
batch_size 512
optimizer adam
adam_lr 0.0001
sgd_lr 0.0001
l2_regularization 0
num_negative 1
drop_ratio 0.5
mlp_layers [64, 128, 64, 32, 16]
verbose 100
feature_pre False
gnn GCN
mf NMF
mixed_model both
model_dir checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model
alias test_model_2021_3_24_11_11
gnn_layers [256, 128, 64, 32]
use_cuda True


In [4]:
# load dataset
load_path = '../input/sample_data_{}_{}_users_{}_{}.pkl'

train_friends = pd.read_pickle(load_path.format(data_name, num_sample, 'train_pos', ratio_sample_pos))
friends = pd.read_pickle(load_path.format(data_name, num_sample, 'friends', ratio_sample_pos))
df_valid = pd.read_pickle(load_path.format(data_name, num_sample, 'valid', ratio_sample_pos))
df_test = pd.read_pickle(load_path.format(data_name, num_sample, 'test', ratio_sample_pos))
G_user = nx.read_gpickle(load_path.format(data_name, num_sample, 'train_graph', ratio_sample_pos))
# features
array_adj = pd.read_pickle(load_path.format(data_name, num_sample, 'array_adj', ratio_sample_pos))

In [5]:
df_non_train = pd.concat([df_valid, df_test]).reset_index(drop=True)
self_friend = pd.DataFrame({'u1': list(G_user.nodes()),
                            'u2': list(G_user.nodes()),
                            'label': 1})
friends = pd.concat([friends, self_friend]).sort_values(['u1', 'u2'],
                                                    ascending=[True, True]).\
                                                    reset_index(drop=True)
friends = friends[['u1', 'u2', 'label']]
train_negatives = sample_negative(G_user, ratings=friends, test_samples=df_non_train, num_negatives=num_negatives)
del friends

valid_user_left = df_valid['u1'].values.tolist()
valid_user_right = df_valid['u2'].values.tolist()
valid_labels = df_valid['label'].values.tolist()

test_user_left = df_test['u1'].values.tolist()
test_user_right = df_test['u2'].values.tolist()
test_labels = df_test['label'].values.tolist()

In [6]:
features_friends = deepcopy(array_adj.toarray())
user_features = deepcopy(features_friends)

features = torch.FloatTensor(user_features).to(device)

In [7]:
# add more configs
if mixed_model in ['both', 'GNN', 'Simple_GNN']:
    if model_config['gnn_layers'][0] != features.shape[1]:
        model_config['gnn_layers'] = [features.shape[1]] + model_config['gnn_layers'][1:]

In [8]:
# prepare edge_index for torch_geometric
l1 = [list(item) for item in list(G_user.edges)]
l2 = [item[::-1] for item in l1]
l1.extend(l2)

edge_index = torch.tensor(l1, dtype=torch.long)
data = torch_geometric.data.Data(x=features, edge_index=edge_index.t().contiguous()).to(device)

del features
del edge_index

In [9]:
def weights_init(m):
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal(m.weight)

test_user_left = torch.LongTensor(test_user_left).to(device)
test_user_right = torch.LongTensor(test_user_right).to(device)

model = NeuLP(config=model_config, in_dim=data.x.size(1))
model = model.to(device)
model.apply(weights_init)
print(model)

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=model_config['adam_lr'],
                             weight_decay=model_config['l2_regularization'])

In [12]:
ls_train_loss = []
ls_test_auc = []

valid_results = []
test_results = []
for epoch_id, epoch in enumerate(range(model_config['num_epoch'])):
    start_epoch = time.time()
    print('Epoch {} starts !'.format(epoch_id))
    print('-' * 80)
    total_loss = 0
    
    print('is generating train loader...')
    train_loader = instance_a_train_loader(
                    train_ratings=train_friends, 
                    negatives=train_negatives, 
                    num_negatives=1, 
                    batch_size=model_config['batch_size'])
    print('There are {} batchs'.format(len(train_loader)))
    for batch_id, batch in enumerate(train_loader):
        model.train()
        user_left_tensor, user_right_tensor, labels_tensor = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        optimizer.zero_grad()
        train_batch_pred = model.forward(data, user_left_tensor, user_right_tensor)

        loss = model.crit(train_batch_pred.view(-1), labels_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.cpu().item()
        
        if model_config['verbose'] != None:
            if batch_id % model_config['verbose'] == 0:
                model.eval()
                
                valid_labels_pred = model.forward(data, valid_user_left, valid_user_right)
                valid_pred = np.array(valid_labels_pred.view(-1).tolist())
                valid_y = np.array(valid_labels)
                
                test_labels_pred = model.forward(data, test_user_left, test_user_right)
                test_pred = np.array(test_labels_pred.view(-1).tolist())
                test_y = np.array(test_labels)
                
                print('[Training Epoch {}] Batch {}, Loss {:.3f},'
                      ' Valid ROC_AUC {:.3f}, Test ROC_AUC {:.3f}'.format(
                          epoch_id, batch_id, loss.item(),
                          roc_auc_score(y_score=valid_pred, y_true=valid_y),
                          roc_auc_score(y_score=test_pred, y_true=test_y)))
        if (epoch_id==0) & (batch_id == 0):
            ls_train_loss.append(loss.item())
            ls_test_auc.append(roc_auc_score(y_score=test_pred, y_true=test_y))
    
    # evaluate epoch
    model.eval()
    valid_labels_pred = model.forward(data, valid_user_left, valid_user_right)
    valid_pred = np.array(valid_labels_pred.view(-1).tolist())
    valid_y = np.array(valid_labels)

    test_labels_pred = model.forward(data, test_user_left, test_user_right)
    test_pred = np.array(test_labels_pred.view(-1).tolist())
    test_y = np.array(test_labels)

    print('[Training Epoch {}] Batch {}, Loss {:.3f},'
          ' Valid ROC_AUC {:.3f}, Test ROC_AUC {:.3f}'.format(
            epoch_id, batch_id, loss.item(),
            roc_auc_score(y_score=valid_pred, y_true=valid_y),
            roc_auc_score(y_score=test_pred, y_true=test_y)))
    
    valid_results.append(roc_auc_score(y_score=valid_pred, y_true=valid_y))
    test_results.append(roc_auc_score(y_score=test_pred, y_true=test_y))
    
    print('best performance is {:.4f} and epoch_id is {}'.format(
        test_results[valid_results.index(max(valid_results))],
        valid_results.index(max(valid_results))))

Epoch 0 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 0] Batch 0, Loss 0.695, Valid ROC_AUC 0.487, Test ROC_AUC 0.503
[Training Epoch 0] Batch 6, Loss 0.680, Valid ROC_AUC 0.742, Test ROC_AUC 0.699
best performance is 0.6987 and epoch_id is 0
Epoch 1 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 1] Batch 0, Loss 0.688, Valid ROC_AUC 0.726, Test ROC_AUC 0.676
[Training Epoch 1] Batch 6, Loss 0.687, Valid ROC_AUC 0.780, Test ROC_AUC 0.722
best performance is 0.7220 and epoch_id is 1
Epoch 2 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 2] Batch 0, Loss 0.680, Valid ROC_AUC 0.752, Test ROC_AUC 0.694
[Training Epoch 2] Batch 6, Loss 0.683, Valid ROC_AUC 0.787, Test ROC

--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 23] Batch 0, Loss 0.529, Valid ROC_AUC 0.840, Test ROC_AUC 0.776
[Training Epoch 23] Batch 6, Loss 0.504, Valid ROC_AUC 0.879, Test ROC_AUC 0.816
best performance is 0.8162 and epoch_id is 23
Epoch 24 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 24] Batch 0, Loss 0.506, Valid ROC_AUC 0.848, Test ROC_AUC 0.786
[Training Epoch 24] Batch 6, Loss 0.516, Valid ROC_AUC 0.880, Test ROC_AUC 0.815
best performance is 0.8145 and epoch_id is 24
Epoch 25 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 25] Batch 0, Loss 0.503, Valid ROC_AUC 0.852, Test ROC_AUC 0.787
[Training Epoch 25] Batch 6, Loss 0.496, Valid ROC_AUC 0.887, Test ROC_AUC 0.

--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 46] Batch 0, Loss 0.368, Valid ROC_AUC 0.864, Test ROC_AUC 0.793
[Training Epoch 46] Batch 6, Loss 0.321, Valid ROC_AUC 0.905, Test ROC_AUC 0.835
best performance is 0.8435 and epoch_id is 41
Epoch 47 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 47] Batch 0, Loss 0.374, Valid ROC_AUC 0.878, Test ROC_AUC 0.808
[Training Epoch 47] Batch 6, Loss 0.362, Valid ROC_AUC 0.907, Test ROC_AUC 0.836
best performance is 0.8435 and epoch_id is 41
Epoch 48 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 48] Batch 0, Loss 0.344, Valid ROC_AUC 0.874, Test ROC_AUC 0.803
[Training Epoch 48] Batch 6, Loss 0.348, Valid ROC_AUC 0.897, Test ROC_AUC 0.

--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 69] Batch 0, Loss 0.277, Valid ROC_AUC 0.884, Test ROC_AUC 0.827
[Training Epoch 69] Batch 6, Loss 0.261, Valid ROC_AUC 0.917, Test ROC_AUC 0.858
best performance is 0.8574 and epoch_id is 56
Epoch 70 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 70] Batch 0, Loss 0.263, Valid ROC_AUC 0.887, Test ROC_AUC 0.828
[Training Epoch 70] Batch 6, Loss 0.265, Valid ROC_AUC 0.926, Test ROC_AUC 0.865
best performance is 0.8655 and epoch_id is 70
Epoch 71 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 71] Batch 0, Loss 0.316, Valid ROC_AUC 0.895, Test ROC_AUC 0.835
[Training Epoch 71] Batch 6, Loss 0.224, Valid ROC_AUC 0.919, Test ROC_AUC 0.

--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 92] Batch 0, Loss 0.221, Valid ROC_AUC 0.918, Test ROC_AUC 0.865
[Training Epoch 92] Batch 6, Loss 0.262, Valid ROC_AUC 0.936, Test ROC_AUC 0.883
best performance is 0.8940 and epoch_id is 91
Epoch 93 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 93] Batch 0, Loss 0.224, Valid ROC_AUC 0.903, Test ROC_AUC 0.852
[Training Epoch 93] Batch 6, Loss 0.206, Valid ROC_AUC 0.940, Test ROC_AUC 0.890
best performance is 0.8940 and epoch_id is 91
Epoch 94 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 94] Batch 0, Loss 0.214, Valid ROC_AUC 0.911, Test ROC_AUC 0.861
[Training Epoch 94] Batch 6, Loss 0.194, Valid ROC_AUC 0.947, Test ROC_AUC 0.

...

--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 908] Batch 0, Loss 0.034, Valid ROC_AUC 0.950, Test ROC_AUC 0.932
[Training Epoch 908] Batch 6, Loss 0.003, Valid ROC_AUC 0.983, Test ROC_AUC 0.965
best performance is 0.9657 and epoch_id is 838
Epoch 909 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 909] Batch 0, Loss 0.008, Valid ROC_AUC 0.953, Test ROC_AUC 0.935
[Training Epoch 909] Batch 6, Loss 0.017, Valid ROC_AUC 0.984, Test ROC_AUC 0.967
best performance is 0.9670 and epoch_id is 909
Epoch 910 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 910] Batch 0, Loss 0.022, Valid ROC_AUC 0.953, Test ROC_AUC 0.937
[Training Epoch 910] Batch 6, Loss 0.019, Valid ROC_AUC 0.981, Test 

[Training Epoch 930] Batch 6, Loss 0.034, Valid ROC_AUC 0.981, Test ROC_AUC 0.966
best performance is 0.9670 and epoch_id is 909
Epoch 931 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 931] Batch 0, Loss 0.011, Valid ROC_AUC 0.951, Test ROC_AUC 0.935
[Training Epoch 931] Batch 6, Loss 0.005, Valid ROC_AUC 0.979, Test ROC_AUC 0.964
best performance is 0.9670 and epoch_id is 909
Epoch 932 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 932] Batch 0, Loss 0.009, Valid ROC_AUC 0.949, Test ROC_AUC 0.934
[Training Epoch 932] Batch 6, Loss 0.045, Valid ROC_AUC 0.981, Test ROC_AUC 0.966
best performance is 0.9670 and epoch_id is 909
Epoch 933 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[T

There are 7 batchs
[Training Epoch 953] Batch 0, Loss 0.021, Valid ROC_AUC 0.950, Test ROC_AUC 0.934
[Training Epoch 953] Batch 6, Loss 0.008, Valid ROC_AUC 0.980, Test ROC_AUC 0.964
best performance is 0.9670 and epoch_id is 909
Epoch 954 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 954] Batch 0, Loss 0.021, Valid ROC_AUC 0.951, Test ROC_AUC 0.934
[Training Epoch 954] Batch 6, Loss 0.014, Valid ROC_AUC 0.983, Test ROC_AUC 0.965
best performance is 0.9670 and epoch_id is 909
Epoch 955 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 955] Batch 0, Loss 0.016, Valid ROC_AUC 0.953, Test ROC_AUC 0.936
[Training Epoch 955] Batch 6, Loss 0.010, Valid ROC_AUC 0.984, Test ROC_AUC 0.966
best performance is 0.9664 and epoch_id is 955
Epoch 956 starts !
-------------------------------

--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 976] Batch 0, Loss 0.008, Valid ROC_AUC 0.951, Test ROC_AUC 0.933
[Training Epoch 976] Batch 6, Loss 0.017, Valid ROC_AUC 0.980, Test ROC_AUC 0.962
best performance is 0.9664 and epoch_id is 955
Epoch 977 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 977] Batch 0, Loss 0.013, Valid ROC_AUC 0.950, Test ROC_AUC 0.933
[Training Epoch 977] Batch 6, Loss 0.006, Valid ROC_AUC 0.980, Test ROC_AUC 0.962
best performance is 0.9664 and epoch_id is 955
Epoch 978 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 978] Batch 0, Loss 0.034, Valid ROC_AUC 0.950, Test ROC_AUC 0.933
[Training Epoch 978] Batch 6, Loss 0.022, Valid ROC_AUC 0.981, Test 

[Training Epoch 998] Batch 6, Loss 0.034, Valid ROC_AUC 0.981, Test ROC_AUC 0.963
best performance is 0.9664 and epoch_id is 955
Epoch 999 starts !
--------------------------------------------------------------------------------
is generating train loader...
There are 7 batchs
[Training Epoch 999] Batch 0, Loss 0.013, Valid ROC_AUC 0.951, Test ROC_AUC 0.933
[Training Epoch 999] Batch 6, Loss 0.011, Valid ROC_AUC 0.981, Test ROC_AUC 0.965
best performance is 0.9664 and epoch_id is 955
